In [1]:
import inspect
from copy import deepcopy, copy
from inspect import Parameter, Signature
from types import FunctionType
from typing import *

from mousse import Dataclass, asdict, asclass

In [2]:
class Foo(Dataclass):
    age: int
    name: str
        
        
class Bar(Dataclass, dynamic=True):
    foo_lst: List[Foo] = []
    foo_dct: Dict[str, Foo] = {}
    foo_nst: List[Dict[str, Foo]] = []

In [3]:
bar = Bar(foo_lst=[
    {
        "age": 10,
        "name": "thu"
    }
], foo_dct={
    'a': {
        "age": 11,
        "name": "ad"
    },
    'b': {
        "age": 12,
        "name": "fd"
    }
}, foo_nst=[
   {
    'a': {
        "age": 11,
        "name": "ad"
    },
    'b': {
        "age": 12,
        "name": "fd"
    }
} 
], foo=Foo(age=14, name="dat"))

In [4]:
bar.foo

Foo(age=14, name="dat")

In [5]:
asdict(bar)

{'foo_lst': [{'age': 10, 'name': 'thu'}],
 'foo_dct': {'a': {'age': 11, 'name': 'ad'}, 'b': {'age': 12, 'name': 'fd'}},
 'foo_nst': [{'a': {'age': 11, 'name': 'ad'}, 'b': {'age': 12, 'name': 'fd'}}],
 'foo': {'age': 14, 'name': 'dat'}}

In [6]:
bar2 = Bar()

In [7]:
asdict(bar2)

{'foo_lst': [], 'foo_dct': {}, 'foo_nst': []}

In [8]:
asdict(b)

NameError: name 'b' is not defined

In [9]:
b.name['name'] = "a"

NameError: name 'b' is not defined

In [10]:
b = deepcopy(bar)

In [11]:
b

Bar(foo_lst=[Foo(age=10, name="thu")], foo_dct={'a': Foo(age=11, name="ad"), 'b': Foo(age=12, name="fd")}, foo_nst=[{'a': Foo(age=11, name="ad"), 'b': Foo(age=12, name="fd")}], foo=Foo(age=14, name="dat"))

In [12]:
asdict(b)

{'foo_lst': [{'age': 10, 'name': 'thu'}],
 'foo_dct': {'a': {'age': 11, 'name': 'ad'}, 'b': {'age': 12, 'name': 'fd'}},
 'foo_nst': [{'a': {'age': 11, 'name': 'ad'}, 'b': {'age': 12, 'name': 'fd'}}],
 'foo': {'age': 14, 'name': 'dat'}}